In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
from diffusers import DDPMScheduler, UNet2DConditionModel
from diffusers import StableDiffusionPipeline, EulerDiscreteScheduler
import os
from torchvision.utils import save_image
import torch.nn.functional as F
import torch.nn as nn

/home/luistorrest/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-04-08 16:59:00.168611: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-08 16:59:01.655355: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    #transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
])


dataset = torchvision.datasets.ImageFolder(root='/home/luistorrest/Documents/Projects/Research/OSTHEO/Model/Dataset/Train', transform=transform)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)


In [4]:
def add_gaussian_noise(images, scheduler, device):
    bsz = images.shape[0]
    timesteps = torch.randint(0, scheduler.config.num_train_timesteps, (bsz,), device=device)
    noise = torch.randn_like(images)
    noisy_images = scheduler.add_noise(images, noise, timesteps)
    return noisy_images, timesteps


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_id = "CompVis/stable-diffusion-v1-4"
scheduler = EulerDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler")

# Create a directory to store the noisy images
if not os.path.exists('noisy_images'):
    os.makedirs('noisy_images')

for i, (images, labels) in enumerate(dataloader):
    images = images.to(device)
    noisy_images, _ = add_gaussian_noise(images, scheduler, device)

    # Save each batch of noisy images
    for j, noisy_image in enumerate(noisy_images):
        save_image(noisy_image, f'noisy_images/noisy_image_{i}_{j}.png')


In [3]:

class Net(nn.Module):
    """
    This class represents a neural network model for image classification.

    Attributes:
        conv1 (nn.Conv2d): First convolutional layer.
        bn1 (nn.BatchNorm2d): Batch normalization layer after the first convolutional layer.
        conv2 (nn.Conv2d): Second convolutional layer.
        bn2 (nn.BatchNorm2d): Batch normalization layer after the second convolutional layer.
        conv3 (nn.Conv2d): Third convolutional layer.
        bn3 (nn.BatchNorm2d): Batch normalization layer after the third convolutional layer.
        conv4 (nn.Conv2d): Fourth convolutional layer.
        bn4 (nn.BatchNorm2d): Batch normalization layer after the fourth convolutional layer.
        dropout1 (nn.Dropout2d): First dropout layer.
        dropout2 (nn.Dropout2d): Second dropout layer.
        fc1 (nn.Linear): First fully connected layer.
        fc2 (nn.Linear): Second fully connected layer.
    """

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(262144, 512)
        self.fc2 = nn.Linear(512, 2)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.bn1(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.bn2(x)
        x = F.max_pool2d(x, 2)
        x = self.conv3(x)
        x = F.relu(x)
        x = self.bn3(x)
        x = self.conv4(x)
        x = F.relu(x)
        x = self.bn4(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output



In [31]:
model = Net().to(device)
criterion = torch.nn.NLLLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

for epoch in range(10):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(dataloader, 0):
        inputs, labels = data
        inputs, _ = add_gaussian_noise(inputs, scheduler, device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print(f'Epoch {epoch + 1}, Loss: {running_loss / (i + 1)}')


/home/luistorrest/.local/lib/python3.10/site-packages/torch/nn/functional.py:1345: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


KeyboardInterrupt: 